In [ ]:
# !pip install transformers -q
# !pip install datasets -q
# !pip install evaluate -q

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
import evaluate
# from datasets import load_dataset

import pandas as pd

df_train = pd.read_csv('/home/aflah20082/NLP_Project/Data/PreprocessedData/train_preprocessed.csv')
df_test = pd.read_csv('/home/aflah20082/NLP_Project/Data/PreprocessedData/test_preprocessed.csv')
df_val = pd.read_csv('/home/aflah20082/NLP_Project/Data/PreprocessedData/val_preprocessed.csv')

df_train = df_train[['preprocessed_text', 'label']]
df_test = df_test[['preprocessed_text']]
df_val = df_val[['preprocessed_text', 'label']]

In [ ]:
df_train = df_train.rename(columns={'preprocessed_text': 'text', 'label': 'label'})
df_test = df_test.rename(columns={'preprocessed_text': 'text'})
df_val = df_val.rename(columns={'preprocessed_text': 'text', 'label': 'label'})

In [ ]:
# Change Label to 0 and 1
df_train['label'] = df_train['label'].replace({'NOT': 1, 'OFF': 0})
df_val['label'] = df_val['label'].replace({'NOT': 1, 'OFF': 0})

In [ ]:
import datasets
from datasets import Dataset, DatasetDict
tds = Dataset.from_pandas(df_train[['text', 'label']], preserve_index=False)
vds = Dataset.from_pandas(df_val[['text', 'label']], preserve_index=False)
testds = Dataset.from_pandas(df_test[['text']], preserve_index=False)

ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds
ds['test'] = testds

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
import numpy as np
metric1 = evaluate.load("accuracy")
metric2 = evaluate.load("precision")
metric3 = evaluate.load("recall")

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) 
training_args = TrainingArguments(output_dir=f"BertBaseUncasedSaves", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
predictions = trainer.predict(tokenized_datasets["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
accuracy = metric1.compute(predictions=preds, references=predictions.label_ids)
precision = metric2.compute(predictions=preds, references=predictions.label_ids)
recall = metric3.compute(predictions=preds, references=predictions.label_ids)
# Write to Txt File
with open(f"/home/aflah20082/NLP_Project/Models/BertBaseUncased.txt", "w") as f:
    f.write(f"Accuracy: {accuracy}\n")
    f.write(f"Precision: {precision}\n")
    f.write(f"Recall: {recall}\n")


In [ ]:
checkpoints = ['500', '1000', '1500', '2000', '2500', '3000', '3500']
ls_results = []
for checkpoint in checkpoints:
    model = AutoModelForSequenceClassification.from_pretrained(f"/home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-{checkpoint}", num_labels=2)
    training_args = TrainingArguments(output_dir=f"/home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-{checkpoint}", evaluation_strategy="epoch")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,
    )
    print(f"Checkpoint: {checkpoint}")
    # Accuracy
    metric = evaluate.load("accuracy")
    # Predictions for Test Data
    predictions = trainer.predict(tokenized_datasets["validation"])
    preds = np.argmax(predictions.predictions, axis=-1)
    accuracy = metric1.compute(predictions=preds, references=predictions.label_ids)
    precision = metric2.compute(predictions=preds, references=predictions.label_ids)
    recall = metric3.compute(predictions=preds, references=predictions.label_ids)
    # Macro F1 Score SKLearn
    from sklearn.metrics import f1_score
    macro_f1 = f1_score(predictions.label_ids, preds, average='macro')
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Macro F1: {macro_f1}")
    ls_results.append([checkpoint, accuracy, precision, recall, macro_f1])

In [ ]:
# print results
df_results = pd.DataFrame(ls_results, columns=['checkpoint', 'accuracy', 'precision', 'recall', 'macro_f1'])
df_results 

In [ ]:
# load Best checkpoint: 3500 Here
model = AutoModelForSequenceClassification.from_pretrained(f"/home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-3500", num_labels=2)
training_args = TrainingArguments(output_dir=f"/home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-3500", evaluation_strategy="epoch", num_train_epochs=1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_eval_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
# Finetune on Validation Data for 1 Epoch
trainer.train()
# Get Predictions
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
ls = []
for i in range(len(preds)):
    ls.append([i, preds[i]])
df = pd.DataFrame(ls, columns=['id', 'label'])

In [ ]:
df